# Exercício - Mini Projeto de Análise de Dados

Vamos fazer um exercício completo de pandas para um miniprojeto de análise de dados.

Esse exercício vai obrigar a gente a usar boa parte dos conhecimento de pandas e até de outros módulos que já aprendemos ao longo do curso.

### O que temos?

Temos os dados de 2019 de uma empresa de prestação de serviços. 

- CadastroFuncionarios
- CadastroClientes
- BaseServiçosPrestados

Obs1: Para ler arquivos csv, temos o read_csv<br>
Obs2: Para ler arquivos xlsx (arquivos em excel normais, que não são padrão csv), temos o read_excel

### O que queremos saber/fazer?

1. Valor Total da Folha Salarial -> Qual foi o gasto total com salários de funcionários pela empresa? <br>
    Sugestão: calcule o salário total de cada funcionário, salário + benefícios + impostos, depois some todos os salários
    
    
2. Qual foi o faturamento da empresa?<br>
    Sugestão: calcule o faturamento total de cada serviço e depois some o faturamento de todos
    
    
3. Qual o % de funcionários que já fechou algum contrato?<br>
    Sugestão: na base de serviços temos o funcionário que fechou cada serviço. Mas nem todos os funcionários que a empresa tem já fecharam algum serviço.<br>
    . Na base de funcionários temos uma lista com todos os funcionários<br>
    . Queremos calcular Qtde_Funcionarios_Fecharam_Serviço / Qtde_Funcionários_Totais<br>
    . Para calcular a qtde de funcionários que fecharam algum serviço, use a base de serviços e conte quantos funcionários tem ali. Mas lembre-se, cada funcionário só pode ser contado uma única vez.<br><br>
    Dica: se você aplicar o método .unique() em uma variável que é apenas 1 coluna de um dataframe, ele vai excluir todos os valores duplicados daquela coluna.<br>
    Ex: unicos_colunaA = dataframe['colunaA'].unique() te dá como resposta uma lista com todos os itens da colunaA aparecendo uma única vez. Todos os valores repetidos da colunaA são excluidos da variável unicos_colunaA 
    
    
4. Calcule o total de contratos que cada área da empresa já fechou


5. Calcule o total de funcionários por área


6. Qual o ticket médio mensal (faturamento médio mensal) dos contratos?<br>
    Dica: .mean() calcula a média -> exemplo: media_colunaA = dataframe['colunaA'].mean()

Obs: Lembrando as opções mais usuais de encoding:<br>
encoding='latin1', encoding='ISO-8859-1', encoding='utf-8' ou então encoding='cp1252'

Observação Importante: Se o seu código der um erro na hora de importar os arquivos:<br>
- CadastroClientes.csv
- CadastroFuncionarios.csv

Use separador ";" (ponto e vírgula) para resolver

In [44]:
import pandas as pd

cadastro_clientes_df = pd.read_csv("CadastroClientes.csv", sep=";", decimal=",")
funcionarios_df = pd.read_csv("CadastroFuncionarios.csv", sep=";", decimal=",")
servicos_df = pd.read_excel("BaseServiçosPrestados.xlsx")

# Parte 1: Calculando o custo total de cada funcionário
quanto_custa_func = funcionarios_df[["ID Funcionário", "Salario Base", "Impostos", "VT", "VR"]]
quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)

# Parte 2: Criando o DataFrame de faturamento
faturamento_df = cadastro_clientes_df.merge(servicos_df, on="ID Cliente", how="inner")
faturamento_df.drop(columns=["Cliente"], inplace=True)
faturamento_df["Faturamento da Empresa"] = faturamento_df["Valor Contrato Mensal"] * faturamento_df["Tempo Total de Contrato (Meses)"]

# Parte 3: Criando o DataFrame que calculará a % de fechamento por funcionário
# Contando quantas vezes cada funcionário aparece na coluna "ID Funcionário" de servicos_df
porcent_func_df = servicos_df["ID Funcionário"].value_counts().reset_index()
porcent_func_df.columns = ["ID Funcionário", "Número de Fechamentos"]

# Juntando com o DataFrame de custo total de cada funcionário para obter a % de fechamento
porcent_func_df = pd.merge(porcent_func_df, quanto_custa_func, on="ID Funcionário", how="left")
porcent_func_df["% de Fechamento"] = (porcent_func_df["Número de Fechamentos"] / porcent_func_df["Número de Fechamentos"].sum()) * 100

# Exibindo tudo
print("Parte 1: Custos totais de cada funcionário")
display(quanto_custa_func)
print("\nParte 2: Faturamento da empresa")
display(faturamento_df)
print("\nParte 3: % de fechamento por funcionário")
display(porcent_func_df)


Parte 1: Custos totais de cada funcionário


C:\Users\Giovani Dantas\AppData\Local\Temp\ipykernel_17284\2921066989.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)


,ID Funcionário,Salario Base,Impostos,VT,VR,Soma Total
0,1,21910,10955.0,242,719.04,33827.04
1,2,5404,2702.0,154,574.56,8836.56
2,3,16066,8033.0,154,729.12,24985.12
3,4,21305,10652.5,220,524.16,32705.66
4,5,5098,2549.0,176,725.76,8553.76
...,...,...,...,...,...,...
109,143,10793,5396.5,242,514.08,17088.58
110,144,4048,2024.0,198,796.32,7210.32
111,148,24596,12298.0,242,561.12,37845.12
112,149,5078,2539.0,308,665.28,8739.28



Parte 2: Faturamento da empresa


,ID Cliente,Valor Contrato Mensal,Codigo do Servico,ID Funcionário,Tempo Total de Contrato (Meses),Faturamento da Empresa
0,1,540,OS0001,67,14,7560
1,2,1260,OS0002,17,12,15120
2,4,2520,OS0003,116,14,35280
3,5,3510,OS0004,37,8,28080
4,6,2340,OS0005,130,8,18720
...,...,...,...,...,...,...
232,315,1575,OS0233,111,4,6300
233,316,3690,OS0234,124,8,29520
234,317,2385,OS0235,72,6,14310
235,319,4185,OS0236,90,14,58590



Parte 3: % de fechamento por funcionário


,ID Funcionário,Número de Fechamentos,Salario Base,Impostos,VT,VR,Soma Total,% de Fechamento
0,47,6,7480,3740.0,264,779.52,12310.52,2.531646
1,84,5,9326,4663.0,220,588.00,14881.00,2.109705
2,37,5,3398,1699.0,242,540.96,5916.96,2.109705
3,4,5,21305,10652.5,220,524.16,32705.66,2.109705
4,67,4,12054,6027.0,308,829.92,19285.92,1.687764
...,...,...,...,...,...,...,...,...
94,5,1,5098,2549.0,176,725.76,8553.76,0.421941
95,45,1,3374,1687.0,286,520.80,5912.80,0.421941
96,102,1,2378,1189.0,220,507.36,4396.36,0.421941
97,10,1,24857,12428.5,264,719.04,38278.54,0.421941


In [41]:
import pandas as pd

cadastro_clientes_df = pd.read_csv("CadastroClientes.csv", sep=";", decimal=",")
funcionarios_df = pd.read_csv("CadastroFuncionarios.csv", sep=";", decimal=",")
servicos_df = pd.read_excel("BaseServiçosPrestados.xlsx")

# Parte 1: Calculando o custo total de cada funcionário
quanto_custa_func = funcionarios_df[["Salario Base", "Impostos", "VT", "VR"]]
quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)

# Parte 2: Criando o DataFrame de faturamento
faturamento_df = cadastro_clientes_df.merge(servicos_df, on="ID Cliente", how="inner")
faturamento_df.drop(columns=["Cliente"], inplace=True)
faturamento_df["Faturamento da Empresa"] = faturamento_df["Valor Contrato Mensal"] * faturamento_df["Tempo Total de Contrato (Meses)"]
#parte 2 concluida 
#parte 3 Criando o Datafrime que irá calcular % de fechamento por funcionário
porcent_func_df = servicos_df["ID Funcionário"].value_counts()




# Exibindo tudo
display(porcent_func_df)
display(quanto_custa_func)
display(servicos_df)
display(cadastro_clientes_df)
display(faturamento_df)


C:\Users\Giovani Dantas\AppData\Local\Temp\ipykernel_17284\4149608650.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)


ID Funcionário
47     6
84     5
37     5
4      5
67     4
      ..
5      1
45     1
102    1
10     1
72     1
Name: count, Length: 99, dtype: int64

,Salario Base,Impostos,VT,VR,Soma Total
0,21910,10955.0,242,719.04,33826.04
1,5404,2702.0,154,574.56,8834.56
2,16066,8033.0,154,729.12,24982.12
3,21305,10652.5,220,524.16,32701.66
4,5098,2549.0,176,725.76,8548.76
...,...,...,...,...,...
109,10793,5396.5,242,514.08,16945.58
110,4048,2024.0,198,796.32,7066.32
111,24596,12298.0,242,561.12,37697.12
112,5078,2539.0,308,665.28,8590.28


,Codigo do Servico,ID Funcionário,ID Cliente,Tempo Total de Contrato (Meses)
0,OS0001,67,1,14
1,OS0002,17,2,12
2,OS0003,116,4,14
3,OS0004,37,5,8
4,OS0005,130,6,8
...,...,...,...,...
232,OS0233,111,315,4
233,OS0234,124,316,8
234,OS0235,72,317,6
235,OS0236,90,319,14


,ID Cliente,Cliente,Valor Contrato Mensal
0,1,Teixeira Gonçalves,540
1,2,Souza Santos,1260
2,3,Emídio Alves,3195
3,4,Santos Costa,2520
4,5,Do Monteiro,3510
...,...,...,...
315,316,Manoel Costa,3690
316,317,Gomes Machado,2385
317,318,Alkindar Cardozo,3510
318,319,Pereira Fazenda,4185


,ID Cliente,Valor Contrato Mensal,Codigo do Servico,ID Funcionário,Tempo Total de Contrato (Meses),Faturamento da Empresa
0,1,540,OS0001,67,14,7560
1,2,1260,OS0002,17,12,15120
2,4,2520,OS0003,116,14,35280
3,5,3510,OS0004,37,8,28080
4,6,2340,OS0005,130,8,18720
...,...,...,...,...,...,...
232,315,1575,OS0233,111,4,6300
233,316,3690,OS0234,124,8,29520
234,317,2385,OS0235,72,6,14310
235,319,4185,OS0236,90,14,58590


In [33]:
import pandas as pd

cadastro_clientes_df = pd.read_csv("CadastroClientes.csv", sep=";", decimal=",")
funcionarios_df = pd.read_csv("CadastroFuncionarios.csv", sep=";", decimal=",")
servicos_df = pd.read_excel("BaseServiçosPrestados.xlsx")

# Parte 1: Calculando o custo total de cada funcionário
quanto_custa_func = funcionarios_df[["Salario Base", "Impostos", "VT", "VR"]]
quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)

# Parte 2: Criando o DataFrame de faturamento
# Mesclar os DataFrames usando o ID Cliente como chave
faturamento_df = cadastro_clientes_df.merge(servicos_df, on="ID Cliente", how="inner")
faturamento_df.drop(columns=["Cliente"], inplace=True)

# Exibindo tudo
display(quanto_custa_func)
display(servicos_df)
display(cadastro_clientes_df)
display(faturamento_df)


C:\Users\Giovani Dantas\AppData\Local\Temp\ipykernel_17284\2168168295.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quanto_custa_func['Soma Total'] = quanto_custa_func.sum(axis=1)


,Salario Base,Impostos,VT,VR,Soma Total
0,21910,10955.0,242,719.04,33826.04
1,5404,2702.0,154,574.56,8834.56
2,16066,8033.0,154,729.12,24982.12
3,21305,10652.5,220,524.16,32701.66
4,5098,2549.0,176,725.76,8548.76
...,...,...,...,...,...
109,10793,5396.5,242,514.08,16945.58
110,4048,2024.0,198,796.32,7066.32
111,24596,12298.0,242,561.12,37697.12
112,5078,2539.0,308,665.28,8590.28


,Codigo do Servico,ID Funcionário,ID Cliente,Tempo Total de Contrato (Meses)
0,OS0001,67,1,14
1,OS0002,17,2,12
2,OS0003,116,4,14
3,OS0004,37,5,8
4,OS0005,130,6,8
...,...,...,...,...
232,OS0233,111,315,4
233,OS0234,124,316,8
234,OS0235,72,317,6
235,OS0236,90,319,14


,ID Cliente,Cliente,Valor Contrato Mensal
0,1,Teixeira Gonçalves,540
1,2,Souza Santos,1260
2,3,Emídio Alves,3195
3,4,Santos Costa,2520
4,5,Do Monteiro,3510
...,...,...,...
315,316,Manoel Costa,3690
316,317,Gomes Machado,2385
317,318,Alkindar Cardozo,3510
318,319,Pereira Fazenda,4185


,ID Cliente,Valor Contrato Mensal,Codigo do Servico,ID Funcionário,Tempo Total de Contrato (Meses)
0,1,540,OS0001,67,14
1,2,1260,OS0002,17,12
2,4,2520,OS0003,116,14
3,5,3510,OS0004,37,8
4,6,2340,OS0005,130,8
...,...,...,...,...,...
232,315,1575,OS0233,111,4
233,316,3690,OS0234,124,8
234,317,2385,OS0235,72,6
235,319,4185,OS0236,90,14
